In [ ]:
#!/usr/bin/env python3
"""
demo_complejidades.py

Script didáctico para mostrar (con ejemplos en Python) y MEDIR tiempos de:
O(1), O(log n), O(n), O(n log n), O(n^2), O(n^3), O(2^n), O(n!)
(+ extras: O(n+m) y O(n·m))

Ejecuta:
    python demo_complejidades.py

Notas para clase:
- En las complejidades explosivas (2^n, n!) NO guardamos resultados: solo contamos/iteramos.
- Ajusta los tamaños si tu máquina es más lenta/rápida.
"""

from __future__ import annotations

from dataclasses import dataclass
from itertools import combinations, permutations
from time import perf_counter
from typing import Callable, Any


# -------------------------
# Utilidades de timing
# -------------------------

def time_call(fn: Callable[[], Any], repeats: int = 3) -> float:
    """Tiempo medio (segundos) de ejecutar fn() 'repeats' veces."""
    t0 = perf_counter()
    for _ in range(repeats):
        fn()
    t1 = perf_counter()
    return (t1 - t0) / repeats


def fmt_s(seconds: float) -> str:
    if seconds < 1e-6:
        return f"{seconds * 1e9:.1f} ns"
    if seconds < 1e-3:
        return f"{seconds * 1e6:.1f} µs"
    if seconds < 1:
        return f"{seconds * 1e3:.1f} ms"
    return f"{seconds:.3f} s"


def header(title: str) -> None:
    print("\n" + "=" * len(title))
    print(title)
    print("=" * len(title))


# -------------------------
# Algoritmos (autoexplicados)
# -------------------------

# O(1)
def last_element(xs: list[int]) -> int:
    """O(1): acceso directo por índice."""
    return xs[-1]


# O(log n)
def binary_search(sorted_xs: list[int], target: int) -> int:
    """O(log n): búsqueda binaria en lista ORDENADA. Devuelve índice o -1."""
    lo, hi = 0, len(sorted_xs) - 1
    while lo <= hi:
        mid = (lo + hi) // 2
        v = sorted_xs[mid]
        if v == target:
            return mid
        if v < target:
            lo = mid + 1
        else:
            hi = mid - 1
    return -1


# O(n)
def linear_contains(xs: list[int], target: int) -> bool:
    """O(n): pertenencia en lista (peor caso recorre todo)."""
    for v in xs:
        if v == target:
            return True
    return False


# O(n log n)
def sort_copy(xs: list[int]) -> list[int]:
    """O(n log n): ordenar (comparación)."""
    return sorted(xs)


# O(n^2)
def count_equal_pairs(xs: list[int]) -> int:
    """
    O(n^2): compara todas las parejas (i<j) y cuenta cuántas son iguales.
    (Ejemplo típico de doble bucle).
    """
    c = 0
    n = len(xs)
    for i in range(n):
        for j in range(i + 1, n):
            if xs[i] == xs[j]:
                c += 1
    return c


# O(n^3)
def triple_loop_work(n: int) -> int:
    """
    O(n^3): hace 'trabajo constante' por cada tripleta (i,j,k).
    Útil como demostración (no es un problema real).
    """
    acc = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                acc += (i + j + k) & 1  # trabajo constante
    return acc


# O(2^n)
def count_subsets(items: list[int]) -> int:
    """
    O(2^n): cuenta subconjuntos sin almacenarlos.
    Total subconjuntos = 2^n.
    """
    c = 0
    n = len(items)
    for r in range(n + 1):
        for _ in combinations(items, r):
            c += 1
    return c


# O(n!)
def count_permutations(items: list[int]) -> int:
    """
    O(n!): cuenta permutaciones sin almacenarlas.
    Total permutaciones = n!.
    """
    c = 0
    for _ in permutations(items):
        c += 1
    return c


# O(n + m)
def sum_two_lists(a: list[int], b: list[int]) -> int:
    """O(n+m): dos recorridos independientes."""
    s = 0
    for x in a:
        s += x
    for y in b:
        s += y
    return s


# O(n·m)
def count_cartesian_pairs(a: list[int], b: list[int]) -> int:
    """O(n·m): doble bucle con dos entradas (producto cartesiano)."""
    c = 0
    for _ in a:
        for _ in b:
            c += 1
    return c


# -------------------------
# Experimentos
# -------------------------

@dataclass
class Experiment:
    name: str
    complexity: str
    sizes: list[int]
    make_case: Callable[[int], Callable[[], Any]]
    repeats_for_n: Callable[[int], int]


def run_experiment(exp: Experiment) -> None:
    header(f"{exp.name}  ({exp.complexity})")
    print("n | tiempo")
    print("--+----------------")
    for n in exp.sizes:
        fn = exp.make_case(n)
        repeats = exp.repeats_for_n(n)
        t = time_call(fn, repeats=repeats)
        print(f"{n:2d} | {fmt_s(t)}")


def main() -> None:
    print("Demo de complejidades (tiempos aproximados; dependen de la CPU).")
    print("Sugerencia: en clase, aumenta n poco a poco para ver el 'salto'.")

    # Casos base
    # (usamos datos deterministas para estabilidad)
    def make_list(n: int) -> list[int]:
        return [(i * 17) % 997 for i in range(n)]

    # Experimentos (tamaños elegidos para que, normalmente, termine en portátil)
    experiments: list[Experiment] = [
        Experiment(
            name="Acceso directo: último elemento",
            complexity="O(1)",
            sizes=[10, 1_000, 1_000_000],
            make_case=lambda n: (lambda: last_element(make_list(n))),
            repeats_for_n=lambda n: 2000 if n <= 1000 else 200,
        ),
        Experiment(
            name="Búsqueda binaria en lista ordenada",
            complexity="O(log n)",
            sizes=[1_000, 10_000, 100_000, 1_000_000],
            make_case=lambda n: (
                lambda: binary_search(list(range(n)), n - 1)  # siempre lo encuentra
            ),
            repeats_for_n=lambda n: 1000,
        ),
        Experiment(
            name="Pertenencia lineal en lista (peor caso)",
            complexity="O(n)",
            sizes=[1_000, 10_000, 50_000, 100_000],
            make_case=lambda n: (
                lambda: linear_contains(make_list(n), -1)  # -1 no está => peor caso
            ),
            repeats_for_n=lambda n: 50 if n >= 50_000 else 200,
        ),
        Experiment(
            name="Ordenación de lista",
            complexity="O(n log n)",
            sizes=[1_000, 10_000, 50_000, 100_000],
            make_case=lambda n: (lambda: sort_copy(make_list(n))),
            repeats_for_n=lambda n: 20 if n >= 50_000 else 50,
        ),
        Experiment(
            name="Doble bucle: contar parejas iguales",
            complexity="O(n^2)",
            sizes=[200, 400, 800, 1200],
            make_case=lambda n: (lambda: count_equal_pairs(make_list(n))),
            repeats_for_n=lambda n: 3 if n >= 800 else 5,
        ),
        Experiment(
            name="Triple bucle: trabajo por tripleta",
            complexity="O(n^3)",
            sizes=[20, 30, 40, 50],
            make_case=lambda n: (lambda: triple_loop_work(n)),
            repeats_for_n=lambda n: 3,
        ),
        Experiment(
            name="Explosiva: contar subconjuntos",
            complexity="O(2^n)",
            sizes=[10, 15, 18, 20, 22],
            make_case=lambda n: (lambda: count_subsets(list(range(n)))),
            repeats_for_n=lambda n: 1,
        ),
        Experiment(
            name="Explosiva: contar permutaciones",
            complexity="O(n!)",
            sizes=[7, 8, 9, 10, 11],
            make_case=lambda n: (lambda: count_permutations(list(range(n)))),
            repeats_for_n=lambda n: 1,
        ),
        Experiment(
            name="Dos entradas: sumar dos listas",
            complexity="O(n+m)",
            sizes=[1_000, 10_000, 100_000, 300_000],
            make_case=lambda n: (lambda: sum_two_lists(make_list(n), make_list(n))),
            repeats_for_n=lambda n: 50 if n <= 10_000 else 10,
        ),
        Experiment(
            name="Dos entradas: producto cartesiano (contar pares)",
            complexity="O(n·m)",
            sizes=[200, 400, 800, 1200],
            make_case=lambda n: (lambda: count_cartesian_pairs(make_list(n), make_list(n))),
            repeats_for_n=lambda n: 2 if n >= 800 else 3,
        ),
    ]

    for exp in experiments:
        run_experiment(exp)

    header("Lectura rápida para explicar en clase")
    print("- O(1): el tiempo casi no cambia con n.")
    print("- O(log n): crece muy lento (dividir entre 2 en cada paso).")
    print("- O(n): crece proporcional (una pasada).")
    print("- O(n log n): típico de ordenar (crece más que lineal).")
    print("- O(n^2), O(n^3): comparar 'todas las parejas' / 'todas las tripletas'.")
    print("- O(2^n): cada elemento duplica combinaciones (sí/no).")
    print("- O(n!): todos los órdenes posibles (permutaciones); crece brutal.")
    print("\nSi algo tarda demasiado, baja el último n de esa sección y re-ejecuta.")


if __name__ == "__main__":
    main()